<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_Cpp_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

Solución implementada utilizando un *aproach* funcional, priorizando la inmutabilidad de los datos

In [31]:
%%writefile matrix_mult.cpp
#include <iostream>
#include <vector>
#include <string>
#include <random>
#include <utility>
#include <thread>
#include <stdlib.h>

#define MIN_MATRIX_SIZE 5
#define MAX_MATRIX_SIZE 20
#define MIN_MATRIX_CELL_VALUE -32
#define MAX_MATRIX_CELL_VALUE 32

typedef int t_matrix;
typedef std::vector<std::vector<t_matrix>> Matrix;

int ValidateArgs(int& argc, char* argv[]);
Matrix MatrixGenerator(int size, bool populate = true);
Matrix MatrixGenerator(int row_size, int col_size, bool populate = true);
Matrix SecuentialMult(const Matrix&, const Matrix&);
Matrix ConcurrentMult(const Matrix&, const Matrix&);
void PrintMatrix(const std::string alias, const Matrix&);

int main(int argc, char* argv[])
{
    int size = ValidateArgs(argc, argv);

    Matrix matrix_a  {MatrixGenerator(size)};
    Matrix matrix_b  {MatrixGenerator(size)};
    Matrix matrix_cs {SecuentialMult(matrix_a, matrix_b)};
    Matrix matrix_ch {ConcurrentMult(matrix_a, matrix_b)};

    PrintMatrix("A", matrix_a);
    PrintMatrix("B", matrix_b);
    PrintMatrix("CS", matrix_cs);
    PrintMatrix("CH", matrix_ch);
    std::cout << "Las matrices CS y CH son iguales? "
        // El operador == en vectores compara todos los elementos del vector
        << (matrix_cs == matrix_ch ? "Si" : "No")
        << std::endl;

    return EXIT_SUCCESS;
}

// Valida los argumentos y devuelve el tamaño de la matrices a crear
int ValidateArgs(int& argc, char* argv[])
{
    if (argc != 2)
    {
        std::cout << "Cantidad de argumentos incorrectos" << std::endl;
        exit(EXIT_FAILURE);
    }

    int size = atoi(argv[1]);
    if (size < MIN_MATRIX_SIZE || size > MAX_MATRIX_SIZE)
    {
        std::cout << "El tamaño de la matriz debe estar entre "
            << MIN_MATRIX_SIZE << " y " << MAX_MATRIX_SIZE << std::endl;

        exit(EXIT_FAILURE);
    }

    return size;
}

// Genera una matriz de igual tamaño de filas y columnas
Matrix MatrixGenerator(int size, bool populate)
{
    // "Movemos" al objeto en vez de realizar una copia
    return std::move(MatrixGenerator(size, size, populate));
}

// Genera una matriz y las llena de datos de ser necesario
Matrix MatrixGenerator(int row_size, int col_size, bool populate)
{
    Matrix aux(row_size, std::vector<t_matrix>(col_size));

    if (populate)
    {
        std::random_device rd; // Obtenemos un número random del hardware
        std::mt19937 gen(rd()); // Generamos la semilla (seed)
        std::uniform_int_distribution<> distr(MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE); // Definimos el rango

        for (int i=0; i < row_size; ++i)
            for (int j=0; j < col_size; ++j)
                aux[i][j] = distr(gen);
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera secuencial
Matrix SecuentialMult(const Matrix& a, const Matrix& b)
{
    Matrix aux = MatrixGenerator(a.size(), false);

    for (int i=0; i < a.size(); ++i)
        for (int j=0; j < b[0].size(); ++j)
            for (int k=0; k < a.size(); ++k)
                aux[i][j] += a[i][k] * b[k][j];    

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera concurrente
Matrix ConcurrentMult(const Matrix& a, const Matrix& b)
{
    Matrix aux = MatrixGenerator(a.size(), false);
    std::vector<std::thread> workers;

    for (int i; i < a.size(); ++i)
    {
        workers.push_back(std::thread(
                // Función anónima o lambda que realiza la asignación y multiplicación
                [&aux, &a, &b, i]()
                {
                    for (int j=0; j < b[0].size(); ++j)
                        for (int k=0; k < a.size(); ++k)
                            aux[i][j] += a[i][k] * b[k][j];
                }
            )
        );
    }
    
    for(auto& worker : workers)
    {
        worker.join();
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Imprime la matriz con su alias
void PrintMatrix(const std::string alias, const Matrix& m)
{
    std::cout << "Matrix " << alias << ":" << std::endl;
    
    for(const auto& row : m)
    {
        for(const auto& col : row)
        {
            std::cout << "\t" << col << "\t";
        }
        std::cout << std::endl;
    }
}


Overwriting matrix_mult.cpp


Compilamos el programa, indicando que utilice el módulo pthread para poder utilizar hilos

In [32]:
!g++ matrix_mult.cpp -pthread -std=c++14 -o matrix_mult.out

## Pruebas de validación de parametros:

In [33]:
!./matrix_mult.out

Cantidad de argumentos incorrectos


In [34]:
!./matrix_mult.out 1

El tamaño de la matriz debe estar entre 5 y 20


In [35]:
!./matrix_mult.out 21

El tamaño de la matriz debe estar entre 5 y 20


## Ejecución exitosa:

In [36]:
!./matrix_mult.out 6

Matrix A:
	-24		-28		28		0		-10		15	
	8		19		28		22		12		-10	
	-8		-15		-1		4		1		18	
	-16		-10		5		31		2		-27	
	32		-20		-9		-22		9		32	
	12		32		-28		-4		-5		21	
Matrix B:
	-12		-28		10		-8		-8		-19	
	9		-13		-21		-11		-14		26	
	-4		19		-21		27		7		-28	
	32		13		-14		-32		-24		-25	
	10		-16		1		-11		11		12	
	-21		-18		-16		-27		28		13	
Matrix CS:
	-491		1458		-490		961		1090		-981	
	997		335		-1043		-83		-810		-978	
	-275		112		-87		-423		686		-64	
	1661		1530		-55		88		-1175		-1198	
	-1814		-1813		734		-538		1484		198	
	-363		-1634		-249		-1588		-111		1701	
Matrix CH:
	-491		1458		-490		961		1090		-981	
	997		335		-1043		-83		-810		-978	
	-275		112		-87		-423		686		-64	
	1661		1530		-55		88		-1175		-1198	
	-1814		-1813		734		-538		1484		198	
	-363		-1634		-249		-1588		-111		1701	
Las matrices CS y CH son iguales? Si
